In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from src.data.dataloader import OHLDataModule

import pytorch_lightning as pl
from lightning_transformers.task.nlp.text_classification import (
    TextClassificationDataModule,
    TextClassificationTransformer,
)

In [3]:
dataset_df = pd.read_csv('../data/processed/dataset.csv')
dataset_df.head()

,question,label,question_type
0,LYUMJEV,benefit_search-pharmacy-1,0
1,Which plans cover LYUMJEV and what is the pric...,benefit_search-pharmacy-1,0
2,Which plan offers lyumjev and price list for it,benefit_search-pharmacy-1,0
3,Which plan has lyumjev coverage and this cost,benefit_search-pharmacy-1,0
4,Which plans offer lyumjev coverage and is it f...,benefit_search-pharmacy-1,0


In [8]:
l_encoder = LabelEncoder()
l_encoder = l_encoder.fit(dataset_df['label'].values)

In [15]:
l_encoder.transform(dataset_df['label'])

array([185, 185, 185, ..., 295, 295, 295])

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
def preprocess_function(text):
    return tokenizer.encode_plus(text, truncation=True)
preprocess_function('i wan%l to ta1k to someone at company')

{'input_ids': [101, 1045, 14071, 1003, 1048, 2000, 11937, 2487, 2243, 2000, 2619, 2012, 2194, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [2]:
datamodule = OHLDataModule(data_path='../data/processed/dataset.csv',
                           max_length=512,
                           train_bs=32,
                           valid_bs=32,
                           test_size=0.3,
                           model_name='distilbert-base-uncased',
                           aug=True)
datamodule.setup()

Loading data ...
208119 train questions
89195 validation questions


In [3]:
datamodule.train_ds.texts[0], datamodule.train_ds.__getitem__(0).keys()

('Which plan has osmolex 322 mg daily dosing coverage and are this free?',
 dict_keys(['input_ids', 'attention_mask', 'labels']))

In [5]:
dataloader = datamodule.train_dataloader()

In [9]:
next(dataloader._get_iterator())['input_ids'].shape

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the enviro

torch.Size([32, 512])

In [7]:
model = TextClassificationTransformer(pretrained_model_name_or_path="bert-base-uncased", num_labels=datamodule.num_classes)
# model.num_classes = 
trainer = pl.Trainer(accelerator="auto", devices="auto", max_epochs=1)

trainer.fit(model, datamodule)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Loading data ...
208119 train questions
89195 validation questions


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/andrey/roonyx_projects/question-matching/.env/lib/python3.8/site-packages/lightning_transformers/core/model.py:85: UserWarning: You haven't specified an optimizer or lr scheduler. Defaulting to AdamW with an lr of 1e-5 and linear warmup for 10% of steps. To change this, override ``configure_optimizers`` in  TransformerModule.
  rank_zero_warn(
Loading `train_dataloader` to estimate number of stepping batches.

  | Name   | Type                          | Params
---------------------------------------------------------
0 | model  | BertForSequenceClassification | 109 M 
1 | prec   | Precision                     | 0     
2 | recall | Recall                        | 0     
3 | acc    | Accuracy                      | 0     
---------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
438.839   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

ValueError: Expected input batch_size (32) to match target batch_size (9472).

In [5]:
9472/32

296.0